In [1]:
from astropy.table import Table, MaskedColumn
from astropy.time import Time
import astropy.io.ascii
import numpy as np

In [2]:
traw = Table.read('../data/asset1_transactions.csv', format='ascii.csv', converters={
                  'quantity': [astropy.io.ascii.convert_numpy(np.float64)],
                  'unit price': [astropy.io.ascii.convert_numpy(np.float64)],
                  'total price': [astropy.io.ascii.convert_numpy(np.float64)]})

In [3]:
traw['date'] = Time(traw['date'])
traw.rename_column('date', 'dateS')
traw.add_column(MaskedColumn(traw['dateS'], name='dateE', mask=True))

In [4]:
traw.sort(keys=['dateS',])

In [12]:
packets = traw[traw['type'] == 'buy']
packets.remove_columns(['type','total price'])
packets.rename_columns(['unit price'],['priceS'])
packets.add_column(MaskedColumn(np.zeros(len(packets)), name='priceE', mask=True))

In [13]:
packets

dateS,asset,quantity,priceS,dateE,priceE
object,str2,float64,float64,object,float64
2019-01-01 00:00:00.000,A1,5.0,2.0,--,--
2020-01-01 00:00:00.000,A1,10.0,3.0,--,--


In [14]:
selltable = traw[traw['type'] == 'sell']

In [15]:
traw

dateS,asset,type,quantity,unit price,total price,dateE
object,str2,str4,float64,float64,float64,object
2019-01-01 00:00:00.000,A1,buy,5.0,2.0,10.0,--
2020-01-01 00:00:00.000,A1,buy,10.0,3.0,30.0,--
2021-01-01 00:00:00.000,A1,sell,1.0,5.0,5.0,--


In [16]:
selltable

dateS,asset,type,quantity,unit price,total price,dateE
object,str2,str4,float64,float64,float64,object
2021-01-01 00:00:00.000,A1,sell,1.0,5.0,5.0,--


In [17]:
eps = 1e-6
for sale in selltable:
    i_consider = np.searchsorted(packets['dateS'], sale['dateS'], side='right')
    i_unclosed = np.flatnonzero(packets['dateE'].mask[:i_consider])
    saleUnits = sale['quantity']
    ii = 0
    while saleUnits > 0.:
        if len(i_unclosed) <= ii:
            raise ValueError("No match found for sale.")
        irec = i_unclosed[ii]
        packets['dateE'].mask[irec] = False        
        packets['dateE'][irec] = sale['dateS']
        packets['priceE'][irec] = sale['unit price']
        if saleUnits > packets['quantity'][irec]:
            saleUnits -= packets['quantity'][irec] # Still to match sale
        else:
            r = packets['quantity'][irec] - saleUnits
            packets.insert_row(irec+1, packets[irec])
            packets['quantity'][irec] = saleUnits
            irec += 1
            packets['quantity'][irec] = r
            packets['dateE'].mask[irec] = True
            packets['priceE'].mask[irec] = True
            saleUnits = 0.
        ii += 1

In [18]:
print(packets)

         dateS          asset quantity priceS          dateE          priceE
----------------------- ----- -------- ------ ----------------------- ------
2019-01-01 00:00:00.000    A1      1.0    2.0 2021-01-01 00:00:00.000    5.0
2019-01-01 00:00:00.000    A1      4.0    2.0                      --     --
2020-01-01 00:00:00.000    A1     10.0    3.0                      --     --
